In [2]:
using Ripserer
using Eirene
using PersistenceDiagrams
using JLD2
using CSV
using Images
include("extension_method.jl")
     

┌ Warning: `PlotlyBase` 0.5.4 is not compatible with this version of `Plots`. The declared compatibility is 0.7 - 0.8.
└ @ Plots C:\Users\jaydh\.julia\packages\Plots\QZRtR\src\backends.jl:55


Main.ext

In [3]:
# VR
location_tumour_h0 = "persistence_diagrams\\vr\\tumour_h0\\"
location_tumour_h1 = "persistence_diagrams\\vr\\tumour_h1\\"
location_macrophage_h0 = "persistence_diagrams\\vr\\macrophages_h0\\"
location_macrophage_h1 = "persistence_diagrams\\vr\\macrophages_h1\\"

# Dowker
location_mt_h0 = "persistence_diagrams\\dowker\\macrophage_tumour_h0\\"
location_mt_h1 = "persistence_diagrams\\dowker\\macrophage_tumour_h1\\"
location_mv_h0 = "persistence_diagrams\\dowker\\macrophage_vessel_h0\\"
location_mv_h1 = "persistence_diagrams\\dowker\\macrophage_vessel_h1\\"
location_tv_h0 = "persistence_diagrams\\dowker\\tumour_vessel_h0\\"
location_tv_h1 = "persistence_diagrams\\dowker\\tumour_vessel_h1\\"

"persistence_diagrams\\dowker\\tumour_vessel_h1\\"

In [4]:
function save_diagram(path,diagram)
    birth_death_pairs = [(diagram[i][1], diagram[i][2]) for i = 1:length(diagram)]
    save_object(path,birth_death_pairs)
end
function load_diagram(path, dimension)
    birth_death_pairs = load_object(path)
    return PersistenceDiagram(birth_death_pairs;dim = dimension)
end
function tuple2mat(pc)
    mat = zeros(Float64,length(pc),2)
    for i = 1:length(pc)
        mat[i,1] = pc[i][1]
        mat[i,2] = pc[i][2]
    end
    return mat
end
function sim2file(sim)
    id = sim[1]
    time = sim[2]
    return datapath*"\\ID-"*string(id)*string("_")*"time-"*string(time)*"_From2ParamSweep_Data.csv"   
end

sim2file (generic function with 1 method)

In [5]:
datapath = "C:\\Users\\jaydh\\Documents\\dissertation\\29072022\\29072022"

"C:\\Users\\jaydh\\Documents\\dissertation\\29072022\\29072022"

In [6]:
sims = load("sim_ids_and_times.jld2", "sims");

In [7]:
distinct_cell_types = ["Tumour","Macrophage","Vessel"]

for sim in sims
    
    id = sim[1]
    time = sim[2]
    A = string(id)*"_"*string(time)

    df = CSV.read(sim2file(sim))
    cell_coordinates = Dict{String, Array{Tuple{Float64, Float64}, 1}}()
    for cell_type in distinct_cell_types
        # Filter the dataframe for the current cell type
        filtered_df = df[df.celltypes .== cell_type, :]
        
        # Extract x and y coordinates as a 2D array
        coordinates = [(x, y) for (x, y) in zip(filtered_df.points_x, filtered_df.points_y)]
    
        
        # Store the coordinates in the dictionary
        cell_coordinates[cell_type] = coordinates
    end
    M = cell_coordinates["Macrophage"];
    T = cell_coordinates["Tumour"];
    V = cell_coordinates["Vessel"];

    M_dim_max = minimum([length(M) - 2, 1])
    T_dim_max = minimum([length(T) - 2, 1])

    d_M = Nothing
    d_T = Nothing

    if M_dim_max == -2
        # M is empty
        d_M = [PersistenceDiagram([]),PersistenceDiagram([])]
    elseif M_dim_max < 1 
        # M has 1 or 2 points
        d_M = [ripserer(M,dim_max=M_dim_max)[1],PersistenceDiagram([])]
    else
        # M has more than 2 points 
        d_M = ripserer(M)
    end
    
    if T_dim_max == -2
        # T is empty
        d_T = [PersistenceDiagram([]),PersistenceDiagram([])]
    elseif T_dim_max < 1 
        # T has 1 or 2 points
        d_T = [ripserer(T,dim_max=T_dim_max)[1],PersistenceDiagram([])]
    else
        # T has more than 2 points 
        d_T = ripserer(T)
    end
    
    vr_macrophages_h0 = d_M[1]
    vr_macrophages_h1 = d_M[2]

    vr_tumour_h0 = d_T[1]
    vr_tumour_h1 = d_T[2]

    #### Dowker Persistence Diagrams

    M = tuple2mat(M)
    T = tuple2mat(T)
    V = tuple2mat(V)

    dowker_mt_h0 = Nothing
    dowker_mt_h1 = Nothing
    dowker_mv_h0 = Nothing
    dowker_mv_h1 = Nothing
    dowker_tv_h0 = Nothing 
    dowker_tv_h1 = Nothing 
    


    # Macrophage - Tumour
    if size(M)[1] == 0 || size(T)[1] == 0
        dowker_mt_h0 = PersistenceDiagram([])
        dowker_mt_h1 = PersistenceDiagram([])
    else
        D_M_T = Nothing
        if length(M) > length(T)
            D_M_T = Images.ImageDistances.pairwise(Euclidean(), T, M, dims = 1)
        else
            D_M_T = Images.ImageDistances.pairwise(Euclidean(), M, T, dims = 1)
        end
        W_M_T = ext.compute_Witness_persistence(D_M_T, maxdim = 1)
        W_M_T_barcode0 = Eirene.barcode(W_M_T["eirene_output"], dim = 0);
        W_M_T_barcode1 = Eirene.barcode(W_M_T["eirene_output"], dim = 1);
        n0 = size(W_M_T_barcode0,1)
        n1 = size(W_M_T_barcode1,1)
        dowker_mt_h0 = PersistenceDiagram([(W_M_T_barcode0[i,1], W_M_T_barcode0[i,2]) for i = 1:n0],dim=0)
        dowker_mt_h1 = PersistenceDiagram([(W_M_T_barcode1[i,1], W_M_T_barcode1[i,2]) for i = 1:n1],dim=1)
    end


    # Macrophage - Vessel 
    if size(M)[1] == 0 || size(V)[1] == 0
        dowker_mv_h0 = PersistenceDiagram([])
        dowker_mv_h1 = PersistenceDiagram([])
    else
        D_M_V = Nothing
        if length(M) > length(V)
            D_M_V = Images.ImageDistances.pairwise(Euclidean(), V, M, dims = 1)
        else
            D_M_V = Images.ImageDistances.pairwise(Euclidean(), M, V, dims = 1)
        end
        W_M_V = ext.compute_Witness_persistence(D_M_V, maxdim = 1)
        W_M_V_barcode0 = Eirene.barcode(W_M_V["eirene_output"], dim = 0);
        W_M_V_barcode1 = Eirene.barcode(W_M_V["eirene_output"], dim = 1);
        n0 = size(W_M_V_barcode0,1)
        n1 = size(W_M_V_barcode1,1)
        dowker_mv_h0 = PersistenceDiagram([(W_M_V_barcode0[i,1], W_M_V_barcode0[i,2]) for i = 1:n0],dim=0)
        dowker_mv_h1 = PersistenceDiagram([(W_M_V_barcode1[i,1], W_M_V_barcode1[i,2]) for i = 1:n1],dim=1)
    end

    # Tumour - Vessel 
    if size(T)[1] == 0 || size(V)[1] == 0
        dowker_tv_h0 = PersistenceDiagram([])
        dowker_tv_h1 = PersistenceDiagram([])
    else
        D_T_V = Nothing
        if length(T) > length(V)
            D_T_V = Images.ImageDistances.pairwise(Euclidean(), V, T, dims = 1)
        else
            D_T_V = Images.ImageDistances.pairwise(Euclidean(), T, V, dims = 1)
        end
        W_T_V = ext.compute_Witness_persistence(D_T_V, maxdim = 1)
        W_T_V_barcode0 = Eirene.barcode(W_T_V["eirene_output"], dim = 0);
        W_T_V_barcode1 = Eirene.barcode(W_T_V["eirene_output"], dim = 1);
        n0 = size(W_T_V_barcode0,1)
        n1 = size(W_T_V_barcode1,1)
        dowker_tv_h0 = PersistenceDiagram([(W_T_V_barcode0[i,1], W_T_V_barcode0[i,2]) for i = 1:n0],dim=0)
        dowker_tv_h1 = PersistenceDiagram([(W_T_V_barcode1[i,1], W_T_V_barcode1[i,2]) for i = 1:n1],dim=1)
    end

    # uncomment to save

    #save_diagram(location_tumour_h0*A, vr_tumour_h0)
    #save_diagram(location_tumour_h1*A,vr_tumour_h1)
    #save_diagram(location_macrophage_h0*A,vr_macrophages_h0)
    #save_diagram(location_macrophage_h1*A,vr_macrophages_h1)

    #save_diagram(location_mt_h0*A,dowker_mt_h0)
    #save_diagram(location_mt_h1*A,dowker_mt_h1)
    #save_diagram(location_mv_h0*A,dowker_mv_h0)
    #save_diagram(location_mv_h1*A,dowker_mv_h1)
    #save_diagram(location_tv_h0*A,dowker_tv_h0)
    #save_diagram(location_tv_h1*A,dowker_tv_h1)


end